In [178]:
# this script adds CAS and CID lists for mixtures to create odorants.csv from published_data/odorants_raw.csv
import pandas as pd
import numpy as np
import re

In [179]:
# read raw csv cut/paste from online version of manuscript
df = pd.read_csv('published_data\\odorants_raw.csv')

# remove spaces at end of strings
df = df.applymap(lambda s: str(s).replace(u'\xa0', u'') if type(s) == str else s)

#set index to Odor code and remove numerical index column
df.set_index('Odor code', inplace=True)
df.drop(df.columns[0], axis=1, inplace=True)

# remove % from values 'V/V in IPM' column
df['(V/V in IPM)'] = df['(V/V in IPM)'].str[:-1].astype(float)
df.rename({'(V/V in IPM)': '% V/V in IPM'}, axis=1, inplace=True)

#add CID for IPM
df.at['NE','CID'] = 8042

df.head()

,Name,CAS number,% V/V in IPM,CID,Number of subjects,Intensity ratings
Odor code,,,,,,
JX,Laevo-fenchone,7787-20-4,7.6,14525.0,985,62.6305
QB,Isoamyl acetate,123-92-2,25.0,31276.0,985,48.0234
SD,3-Propylidene phthalide,17369-59-4,7.9,5373603.0,985,71.9421
EQ,Cuminaldehyde,122-03-2,13.6,326.0,985,74.0538
AY,Strawberry glycidate 1 (aldehyde C-16),77-83-8,100.0,6501.0,985,56.8213


In [180]:
# mapping for odor codes to CID and CAS
odorants = df.dropna(subset=['CID'])
codeToCID = dict(zip(odorants.index, odorants['CID'].astype('int')))
codeToCAS = dict(zip(odorants.index, odorants['CAS number']))

In [181]:
# functions for adding CAS and CID lists for mixtures
def add_CIDs(row, codeToCID):
    if pd.isnull(row['CID']):
        return [codeToCID[x.strip()] for x in re.search(r'\[.*?\]', row['Name']).group(0).strip('[]').split(',')]       
    else: return int(row['CID'])
    
def add_CASs(row, codeToCAS):
    if pd.isnull(row['CAS number']):
        return [codeToCAS[x.strip()] for x in re.search(r'\[.*?\]', row['Name']).group(0).strip('[]').split(',')]       
    else: return row['CAS number']

In [182]:
# add the lists
df['CID'] = df.apply(lambda row: add_CIDs(row, codeToCID), axis=1)
df['CAS number'] = df.apply(lambda row: add_CASs(row, codeToCAS), axis=1)

df.head()

,Name,CAS number,% V/V in IPM,CID,Number of subjects,Intensity ratings
Odor code,,,,,,
JX,Laevo-fenchone,7787-20-4,7.6,14525,985,62.6305
QB,Isoamyl acetate,123-92-2,25.0,31276,985,48.0234
SD,3-Propylidene phthalide,17369-59-4,7.9,5373603,985,71.9421
EQ,Cuminaldehyde,122-03-2,13.6,326,985,74.0538
AY,Strawberry glycidate 1 (aldehyde C-16),77-83-8,100.0,6501,985,56.8213


In [184]:
# write to file
df.to_csv('published_data\\odorants.csv')